In [1]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession\
    .builder\
    .appName("twitter_silver")\
    .getOrCreate()

/home/airflow/.local/lib/python3.8/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/25 17:50:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df_tweet = spark.read.json("/home/airflow/workspaces/airflow-spark/data/silver/twitter_datascience/tweet")

In [5]:
df_tweet.printSchema()

root
 |-- author_id: string (nullable = true)
 |-- conversation_id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- like_count: long (nullable = true)
 |-- quote_count: long (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- text: string (nullable = true)
 |-- process_date: date (nullable = true)



In [6]:
from pyspark.sql import functions as f

In [10]:
tweet_conversas = df_tweet.alias("tweet")\
                    .groupBy(f.to_date("created_at").alias("created_date"))\
                    .agg(
                        f.countDistinct("author_id").alias("n_tweets"),
                        f.sum("like_count").alias("n_like"),
                        f.sum("quote_count").alias("n_quote"),
                        f.sum("reply_count").alias("n_reply"),
                        f.sum("retweet_count").alias("n_retweet")
                    ).withColumn("weekday", f.date_format("created_date", "E"))

In [11]:
tweet_conversas.show()

+------------+--------+------+-------+-------+---------+-------+
|created_date|n_tweets|n_like|n_quote|n_reply|n_retweet|weekday|
+------------+--------+------+-------+-------+---------+-------+
|  2024-03-24|       6|   255|    282|    321|      230|    Sun|
|  2024-03-25|       4|   201|    272|    170|      182|    Mon|
+------------+--------+------+-------+-------+---------+-------+



In [13]:
tweet_conversas.printSchema()

root
 |-- created_date: date (nullable = true)
 |-- n_tweets: long (nullable = false)
 |-- n_like: long (nullable = true)
 |-- n_quote: long (nullable = true)
 |-- n_reply: long (nullable = true)
 |-- n_retweet: long (nullable = true)
 |-- weekday: string (nullable = true)

